## CompKey竞争性关键词推荐算法设计与实现

阶段一中，我们得到了十个种子关键词： **图片 手机 小说 视频 下载 qq 电影 百度 英语 游戏**  
将与种子关键词有关的搜索数据保存在了文件"seedwords_query.train"中  
在这一阶段，我们主要目标是利用竞争性Comp测度的概率模型，确定竞争性关键字并计算输出竞争度  

算法流程如下：
- 用基于第一阶段的数据获取与种子关键字相关的中介关键字
- 统计出所有包含种子关键字和中介关键字的查询搜索量
- 计算所有中介关键字的权重
- 确定竞争性关键字集合
- 计算所有竞争关键字的竞争度
- 竞争关键字排序输出

### 1. 获取中介关键词

为了后续便于对每个种子关键词进行Comp测度计算，提高算法的性能，将和这十个种子关键词有关的搜索数据分别存在对应的文件下

In [191]:
def seedwords_file(seedword,filename):
    query_data = open('./seedwords_query.train','r',encoding='utf-8')
    result_data = open(filename,'w',encoding='utf-8')
    for line in query_data:
        if seedword in line:
            result_data.write(line)
    query_data.close()
    result_data.close()

In [192]:
seedwords_list=['图片','手机','小说','视频','下载','qq','电影','百度','英语','游戏']
for seedword in seedwords_list:
    filename = './seedword_'+seedword
    seedwords_file(seedword,filename)

得到十个种子关键词对应的搜索数据文件，分别对其进行分词、词频统计，找出每个种子关键词的8个中介关键词

##### jieba分词

In [193]:
# 对十个种子关键词搜索数据文件进行分词,并存储
import jieba
def seedwords_segmentation(seedword):
    seedwords_file = './seedword_'+seedword
    segmentation_file = './seedword_segmentation_'+seedword
    seedwords_data = open(seedwords_file,'r',encoding='utf-8')
    segmentation_data = open(segmentation_file,'w',encoding='utf-8')
    for sentence in seedwords_data:
        sentence = sentence[:-1]
        word_seg = jieba.cut(sentence)
        line_string = '\t'.join(word_seg)+'\n'
        segmentation_data.write(line_string)
    seedwords_data.close()
    segmentation_data.close()

In [194]:
for seedword in seedwords_list:
    seedwords_segmentation(seedword)

##### 词频统计
在词频统计前先进行停用词过滤，去除无意义的词，以免对结果产生干扰  
在输出的前12个高频词中再根据语义和实际意义进一步筛选中介关键词

In [212]:
from collections import Counter

#停用词过滤
stop_word_list=set([' ','的','怎么','我','都','与','你','多少','年','月','什么','及',
                    '和','上','如何','是','吗','有','在','可以','什么','用','了','说',
                    '大','很','版','之','为什么','怎么办','能','玩','级','写',','])
def stop_words_filter(word_list):
    word_cleaned = []
    for word in word_list:
        if word not in stop_word_list:
            word_cleaned.append(word)
    return word_cleaned

def getwordlist(seedword):
    wordlist = [];
    segmentation_file = './seedword_segmentation_'+seedword
    seedwords_data = open(segmentation_file,'r',encoding='utf-8')
    for line in seedwords_data:
        line = line[:-1]
        words = line.split('\t')
        wordlist.extend(words)
    seedwords_data.close()
    return wordlist

# 统计与种子关键词一起出现的词的词频
# 在词频前12中剔除词频最高的种子关键词，先放入候选中介关键词集合中 
def word_frequency_count(seedword):
    word_list = getwordlist(seedword)
    word_list = stop_words_filter(word_list)
    count_result = Counter(word_list)
    midkeywords=[]
    flag = False
    for key, val in count_result.most_common(12):
        #词频最高的一定是种子关键词，不放入midkeywords中
        if flag==True:
            midkeywords.append(key)
        flag=True
    return midkeywords

In [258]:
# 得到种子关键词的候选中介关键词集合
midkeywords_list = []
for seedword in seedwords_list:
    midkeywords_list.append(word_frequency_count(seedword))
midkeywords_list

[['大全', '发型', '唯美', '报价', '价格', '搞笑', '2016', '手机', 'qq', '动漫', '背景图片'],
 ['下载', 'qq', '苹果', '软件', '手机号', '华为', '手机游戏', '设置', '壁纸', '视频', '小米'],
 ['穿越', '好看', '类似', '排行榜', '耽美', '阅读', '重生', '主角', '下载', '小说网', '倾城'],
 ['教学', '在线', '观看', '大全', '手机', '腾讯', '在线视频', '搞笑', '舞蹈', '下载', '视频教程'],
 ['txt', '手机', '全集', '软件', '免费', '游戏', '小说', '百度', '官方', '视频', 'qq'],
 ['手机', '头像', '空间', '号', '飞车', '邮箱', '音乐', '网名', '下载', '好友', '群'],
 ['韩国', '电影网', '完整版', '观看', '全集', '在线', '手机', '大全', '电影院', '好看', '微微一笑'],
 ['云', '吧', '资源', '贴', '下载', '百科', '分享', '云盘', '地图', '翻译', '链接'],
 ['上册', '作文', '答案', '英语单词', '大学', '四级', '英语翻译', '翻译', '2016', '教程', '3'],
 ['下载', '手机游戏', '手机', '大全', '小游戏', '单机游戏', '玩游戏', '好玩', '名字', '橙光', '电脑']]

打印出的最高词频一定是种子关键词，因而在统计词频这一步中，我们可以计算出种子关键词的搜索量$|\{s\}|$,将其存在字典s_query_volume中

In [259]:
def seedword_query_volume(seedword):
    word_list = getwordlist(seedword)
    count_result = Counter(word_list)
    for key, val in count_result.most_common(1):
         return val

s_query_volume = {}
for seedword in seedwords_list:
    s_query_volume[seedword] = seedword_query_volume(seedword)

##### 基于语义的中介关键词筛选
因为Jieba分词的语义性不能保证是准确无误的，为了提高算法的精确度，这里我们根据实际意义在词频前12的候选中介关键词中筛选出8个中介关键词  
比如与种子关键词“百度”一同出现的高频词中有“吧”、“贴”，根据实际意义，我们将“贴吧”作为“百度”的中介关键词  
为了防止$Comp$计算公式中分母为0，我们将一些中介词进行调整得语义范围更大一些

In [260]:
# 处理候选的中介关键词集合，筛选中介关键词

# 种子关键词“图片”的中介关键词筛选
del midkeywords_list[0][-2:]
midkeywords_list[0].remove('报价')

# 种子关键词“手机”的中介关键词筛选
del midkeywords_list[1][-2:]
midkeywords_list[1].remove('手机号')
midkeywords_list[1][5]='游戏'

# 种子关键词“小说”的中介关键词筛选
del midkeywords_list[2][-2:]
midkeywords_list[2].remove('类似')

# 种子关键词“视频”的中介关键词筛选
del midkeywords_list[3][-2:]
midkeywords_list[3].remove('在线视频')

# 种子关键词“下载”的中介关键词筛选
del midkeywords_list[4][-3:]

# 种子关键词“qq”的中介关键词筛选
del midkeywords_list[5][-3:]

# 种子关键词“电影”的中介关键词筛选
del midkeywords_list[6][-3:]
midkeywords_list[6][1]='网'

# 种子关键词“百度”的中介关键词筛选
del midkeywords_list[7][-2:]
midkeywords_list[7].remove('贴')
midkeywords_list[7].remove('吧')
midkeywords_list[7].insert(1,'贴吧')

# 种子关键词“英语”的中介关键词筛选
midkeywords_list[8].remove('英语翻译')
midkeywords_list[8][3]='单词'
midkeywords_list[8].remove('2016')
midkeywords_list[8].remove('3')

# 种子关键词“游戏”的中介关键词筛选
midkeywords_list[9].remove('手机游戏')
midkeywords_list[9].remove('小游戏')
midkeywords_list[9].remove('玩游戏')
midkeywords_list[9][3]='单机'

In [261]:
# 得到10个种子关键词对应中介关键词，存在midkeywords这个字典中
midkeywords=dict(zip(seedwords_list,midkeywords_list))
midkeywords

{'图片': ['大全', '发型', '唯美', '价格', '搞笑', '2016', '手机', 'qq'],
 '手机': ['下载', 'qq', '苹果', '软件', '华为', '游戏', '设置', '壁纸'],
 '小说': ['穿越', '好看', '排行榜', '耽美', '阅读', '重生', '主角', '下载'],
 '视频': ['教学', '在线', '观看', '大全', '手机', '腾讯', '搞笑', '舞蹈'],
 '下载': ['txt', '手机', '全集', '软件', '免费', '游戏', '小说', '百度'],
 'qq': ['手机', '头像', '空间', '号', '飞车', '邮箱', '音乐', '网名'],
 '电影': ['韩国', '网', '完整版', '观看', '全集', '在线', '手机', '大全'],
 '百度': ['云', '贴吧', '资源', '下载', '百科', '分享', '云盘', '地图'],
 '英语': ['上册', '作文', '答案', '单词', '大学', '四级', '翻译', '教程'],
 '游戏': ['下载', '手机', '大全', '单机', '好玩', '名字', '橙光', '电脑']}

### 2. 统计包含种子关键字和中介关键字的查询搜索量

在第一步中得到的十个种子关键词文件中，筛选出种子关键词和对应的中介关键词一起出现的查询搜索，统计联合查询搜索量$|\{sa\}|$

In [262]:
# 统计每个种子关键词和其中介关键词一起出现的搜索量
def query_volume_count(seedword):
    count_list = []
    filename = './seedword_'+seedword
    for midkeyword in midkeywords[seedword]:
        count = 0
        query_data = open(filename,'r',encoding = 'utf-8')
        for setence in query_data:
            if seedword in setence and midkeyword in setence:
                count+=1
        count_list.append(count)
        query_data.close()
    return count_list

In [263]:
# 统计得到的搜索量sa,存储在字典sa_query_volume中
sa_query_volume = {}
for seedword in seedwords_list:
    sa_count_list=[]
    sa_count_list = query_volume_count(seedword)
    sa_count_dict = dict(zip(midkeywords[seedword],sa_count_list))
    sa_query_volume[seedword]=sa_count_dict
sa_query_volume

{'图片': {'大全': 32625,
  '发型': 6634,
  '唯美': 4310,
  '价格': 4997,
  '搞笑': 3691,
  '2016': 3767,
  '手机': 3543,
  'qq': 3303},
 '手机': {'下载': 12284,
  'qq': 11719,
  '苹果': 10196,
  '软件': 9702,
  '华为': 5071,
  '游戏': 8634,
  '设置': 4203,
  '壁纸': 4222},
 '小说': {'穿越': 8162,
  '好看': 6064,
  '排行榜': 5220,
  '耽美': 5381,
  '阅读': 6066,
  '重生': 4897,
  '主角': 5933,
  '下载': 4445},
 '视频': {'教学': 5293,
  '在线': 8292,
  '观看': 4669,
  '大全': 4886,
  '手机': 4453,
  '腾讯': 3294,
  '搞笑': 3103,
  '舞蹈': 3022},
 '下载': {'txt': 34647,
  '手机': 12284,
  '全集': 7653,
  '软件': 4855,
  '免费': 4820,
  '游戏': 5013,
  '小说': 4445,
  '百度': 3318},
 'qq': {'手机': 11719,
  '头像': 10719,
  '空间': 6761,
  '号': 6874,
  '飞车': 3926,
  '邮箱': 3743,
  '音乐': 3025,
  '网名': 3126},
 '电影': {'韩国': 5701,
  '网': 6198,
  '完整版': 4238,
  '观看': 3845,
  '全集': 3739,
  '在线': 4330,
  '手机': 3604,
  '大全': 3844},
 '百度': {'云': 41132,
  '贴吧': 4163,
  '资源': 6697,
  '下载': 3318,
  '百科': 2524,
  '分享': 2359,
  '云盘': 2319,
  '地图': 2215},
 '英语': {'上册': 3575,
  '作文': 3397,
  '

### 3. 确定中介关键字的权重

中介关键词权重的计算公式：$$w_a(k)=\frac{|\{sa\}|}{|\{s\}|}$$  
$|\{s\}|$与$|\{sa\}|$我们在第1、2步中得到了，其对应的搜索量分别存在了字典s_query_volume和sa_query_volume中  
我们根据该公式，计算每个种子关键词对应的中介关键词的权重$w_a(k)$

In [264]:
def count_midkeyword_weight():
    midkeyword_weight={}
    for seedword in seedwords_list:
        s = s_query_volume[seedword]
        weight_dict = {}
        for midkeyword in sa_query_volume[seedword]:
            sa = sa_query_volume[seedword][midkeyword]
            weight = sa/s
            weight_dict[midkeyword] = weight
        midkeyword_weight[seedword] = weight_dict
    return midkeyword_weight

In [265]:
# 将中介关键词的权重结果存储在字典w_midkeyword中
w_midkeyword = dict(count_midkeyword_weight())
w_midkeyword

{'图片': {'大全': 0.11208566952506596,
  '发型': 0.022791611697449428,
  '唯美': 0.014807332893579595,
  '价格': 0.0171675736587511,
  '搞笑': 0.012680711301671064,
  '2016': 0.01294181508355321,
  '手机': 0.012172246042216358,
  'qq': 0.011347707783641161},
 '手机': {'下载': 0.06415559455168379,
  'qq': 0.06120477145483413,
  '苹果': 0.05325060583270661,
  '软件': 0.05067059413386814,
  '华为': 0.02648429013119412,
  '游戏': 0.04509275507646027,
  '设置': 0.021950990223113562,
  '壁纸': 0.022050221442299657},
 '小说': {'穿越': 0.05107666506048223,
  '好看': 0.03794767176265183,
  '排行榜': 0.0326660367086152,
  '耽美': 0.033673552400202755,
  '阅读': 0.03796018748552869,
  '重生': 0.030644747464001653,
  '主角': 0.03712789191421723,
  '下载': 0.027816194093830373},
 '视频': {'教学': 0.033609123292716224,
  '在线': 0.052651964924088974,
  '观看': 0.029646891489456272,
  '大全': 0.031024782997961735,
  '手机': 0.02827534971140475,
  '腾讯': 0.020916012115285706,
  '搞笑': 0.01970321359858274,
  '舞蹈': 0.01918888543181342},
 '下载': {'txt': 0.25677949143

### 4. 确定竞争性关键字集合

竞争性关键字的确定：在含有所有查询数据的文件“link_clean_data.train”中搜索，筛选出不含有种子关键词，但有中介关键词的搜索，存储下来。然后再通过分词，词频统计，确定种子关键词对应的竞争性关键字集合。

In [270]:
# 筛选出不含种子关键词，但含有其中介关键词的搜索数据，存储在相应的文件中
def comkey_words_file(seedword):
    query_data = open('./link_clean_data.train','r',encoding='utf-8')
    filename = './comkey_'+seedword
    result_data = open(filename,'w',encoding='utf-8')
    for line in query_data:
        if seedword not in line:
            for midkeyword in midkeywords[seedword]:
                if midkeyword in line:
                    result_data.write(line)
                    break
    query_data.close()
    result_data.close()
    
for seedword in seedwords_list:
    comkey_words_file(seedword)

对得到的文件，按照中介关键词对搜索内容分词，进行词频统计。  
打印出词频前三的词，词频最高的词一般是中介关键词，再基于语义筛选出竞争性关键词。  
对每个中介关键词，都可确定一个其种子关键词对应的竞争性关键字。                                                  

In [271]:
def get_comkey_words(seedword):
    for midkeyword in midkeywords[seedword]:
        comkey_data = open('./comkey_'+seedword,'r',encoding='utf-8')
        comkeyquery_list = []
        for line in comkey_data:
            if midkeyword in line:
                line = line[:-1]
                word_seg = jieba.cut(line)
                comkeyquery_list.extend(word_seg)
        comkeyquery_list=stop_words_filter(comkeyquery_list)
        count_result = Counter(comkeyquery_list)
        for key, val in count_result.most_common(3):
            if key!=midkeyword:
                print(key,val,end='  ')
        comkey_data.close()

In [272]:
for seedword in seedwords_list:
    print(seedword+':\n')
    get_comkey_words(seedword)

图片:

做法 12907  名字 5374  适合 3002  男生 2078  句子 1495  女生 855  价格表 8434  香烟 1775  视频 3059  大全 1593  时间 6295  最新 4896  下载 12021  qq 11194  手机 10919  头像 10427  手机:

txt 34432  全集 7515  头像 10393  空间 5856  7 7775  6s 2825  下载 3805  哪个 2745  荣耀 5292  p9 2574  下载 4298  大全 3998  路由器 2824  电脑 1455  高清 1021  图片 860  小说:

火线 7135  王者 1631  电影 2128  电视剧 1585  2016 3775  前十名 2852  耽 2050  美文 1935  全文 22595  免费 13154  txt 3433  末世 2227  女主角 3792  男主角 1599  txt 33049  手机 11866  视频:

设计 832  教学计划 565  观看 17400  阅读 4895  在线 17304  电影 3568  图片 31527  做法 12985  下载 11673  qq 11234  游戏 1038  qq 708  图片 3586  大全 1640  教学 529  幼儿 241  下载:

重生 2059  微盘 1187  qq 10909  苹果 9786  电视剧 5397  漫画 3905  手机 6621  哪个 3106  阅读 14470  全文 8616  手机游戏 4107  大全 4030  穿越 8198  好看 6087  云 35225  吧 7197  qq:

下载 11313  苹果 10031  女生 4737  情侣 4085  重生 2172  个人空间 2053  查询 16375  单 11866  侠盗 2050  极品飞车 929  163 898  登陆 736  背景音乐 3270  纯音乐 1051  女生 2213  字 1967  电影:

中国 876  女主播 846  官网 66592  网名 19521  视频 1011  在线 868  在线 18240  视频 42

In [283]:
# 竞争性关键词筛选
# 同时得到|{ka}|的值
ka_list=[[12907,2078,1495,8434,3095,4896,12021,10427],
         [34432,10393,7775,3805,5295,1592,2824,1021],
         [7135,2128,2852,2050,22595,2227,3792,33049],
         [832,17400,17304,31527,11673,1038,1640,29],
         [2059,10909,5397,6621,14470,4107,8198,35225],
         [11313,4737,2172,16375,2050,898,3270,1967],
         [876,66592,1011,18240,7890,18840,11613,31519],
         [11794,11417,2717,34264,205,317,1723,2724],
         [3351,10414,6072,2133,9202,401,3577,2921],
         [34412,11511,31527,967,3883,5253,41,6798]]

comp_list=[['做法','男生','句子','价格表','视频','最新','下载','头像'],
          ['txt','头像','苹果7','下载','荣耀','大全','路由器','高清'],
          ['穿越火线','电影','前十名','耽美','全文','末世','女主角','txt'],
          ['设计','观看','在线','图片','下载','游戏','大全','教学'],
          ['重生','qq','电视剧','手机','阅读','手机游戏','穿越','百度云'],
          ['下载','女生','重生','查询','侠盗','163邮箱','背景音乐','字'],
          ['中国','官网','视频','在线','下载','观看','qq','图片'],
          ['青云志','贴吧','人力资源','txt','糗事','视频','360云盘','中国地图'],
          ['语文','字','阅读','英文单词','大学生','成绩','英文翻译','视频教程'],
          ['txt','qq','图片','单机版','地方','大全','exo','笔记本电脑']]


In [373]:
compkeywords=dict(zip(seedwords_list,comp_list))
compkeywords

{'图片': ['做法', '男生', '句子', '价格表', '视频', '最新', '下载', '头像'],
 '手机': ['txt', '头像', '苹果7', '下载', '荣耀', '大全', '路由器', '高清'],
 '小说': ['穿越火线', '电影', '前十名', '耽美', '全文', '末世', '女主角', 'txt'],
 '视频': ['设计', '观看', '在线', '图片', '下载', '游戏', '大全', '教学'],
 '下载': ['重生', 'qq', '电视剧', '手机', '阅读', '手机游戏', '穿越', '百度云'],
 'qq': ['下载', '女生', '重生', '查询', '侠盗', '163邮箱', '背景音乐', '字'],
 '电影': ['中国', '官网', '视频', '在线', '下载', '观看', 'qq', '图片'],
 '百度': ['青云志', '贴吧', '人力资源', 'txt', '糗事', '视频', '360云盘', '中国地图'],
 '英语': ['语文', '字', '阅读', '英文单词', '大学生', '成绩', '英文翻译', '视频教程'],
 '游戏': ['txt', 'qq', '图片', '单机版', '地方', '大全', 'exo', '笔记本电脑']}

In [361]:
# 每个中介关键词对应的竞争性关键词
compwords=dict.fromkeys(seedwords_list,{})
i=0
for list in midkeywords_list: 
    compwords[seedwords_list[i]]=dict(zip(list,comp_list[i]))
    i+=1

{'图片': {'大全': '做法',
  '发型': '男生',
  '唯美': '句子',
  '价格': '价格表',
  '搞笑': '视频',
  '2016': '最新',
  '手机': '下载',
  'qq': '头像'},
 '手机': {'下载': 'txt',
  'qq': '头像',
  '苹果': '苹果7',
  '软件': '下载',
  '华为': '荣耀',
  '游戏': '大全',
  '设置': '路由器',
  '壁纸': '高清'},
 '小说': {'穿越': '穿越火线',
  '好看': '电影',
  '排行榜': '前十名',
  '耽美': '耽美',
  '阅读': '全文',
  '重生': '末世',
  '主角': '女主角',
  '下载': 'txt'},
 '视频': {'教学': '设计',
  '在线': '观看',
  '观看': '在线',
  '大全': '图片',
  '手机': '下载',
  '腾讯': '游戏',
  '搞笑': '大全',
  '舞蹈': '教学'},
 '下载': {'txt': '重生',
  '手机': 'qq',
  '全集': '电视剧',
  '软件': '手机',
  '免费': '阅读',
  '游戏': '手机游戏',
  '小说': '穿越',
  '百度': '百度云'},
 'qq': {'手机': '下载',
  '头像': '女生',
  '空间': '重生',
  '号': '查询',
  '飞车': '侠盗',
  '邮箱': '163邮箱',
  '音乐': '背景音乐',
  '网名': '字'},
 '电影': {'韩国': '中国',
  '网': '官网',
  '完整版': '视频',
  '观看': '在线',
  '全集': '下载',
  '在线': '观看',
  '手机': 'qq',
  '大全': '图片'},
 '百度': {'云': '青云志',
  '贴吧': '贴吧',
  '资源': '人力资源',
  '下载': 'txt',
  '百科': '糗事',
  '分享': '视频',
  '云盘': '360云盘',
  '地图': '中国地图'},
 '英语': {'上册': '语文',


In [312]:
# 将|{ka}|的值存在字典ka_query_list中
ka_query_volume=dict.fromkeys(seedwords_list,{})
i=0
for list in midkeywords_list: 
    ka_query_volume[seedwords_list[i]]=dict(zip(list,ka_list[i]))
    i+=1
ka_query_volume

{'图片': {'大全': 12907,
  '发型': 2078,
  '唯美': 1495,
  '价格': 8434,
  '搞笑': 3095,
  '2016': 4896,
  '手机': 12021,
  'qq': 10427},
 '手机': {'下载': 34432,
  'qq': 10393,
  '苹果': 7775,
  '软件': 3805,
  '华为': 5295,
  '游戏': 1592,
  '设置': 2824,
  '壁纸': 1021},
 '小说': {'穿越': 7135,
  '好看': 2128,
  '排行榜': 2852,
  '耽美': 2050,
  '阅读': 22595,
  '重生': 2227,
  '主角': 3792,
  '下载': 33049},
 '视频': {'教学': 832,
  '在线': 17400,
  '观看': 17304,
  '大全': 31527,
  '手机': 11673,
  '腾讯': 1038,
  '搞笑': 1640,
  '舞蹈': 29},
 '下载': {'txt': 2059,
  '手机': 10909,
  '全集': 5397,
  '软件': 6621,
  '免费': 14470,
  '游戏': 4107,
  '小说': 8198,
  '百度': 35225},
 'qq': {'手机': 11313,
  '头像': 4737,
  '空间': 2172,
  '号': 16375,
  '飞车': 2050,
  '邮箱': 898,
  '音乐': 3270,
  '网名': 1967},
 '电影': {'韩国': 876,
  '网': 66592,
  '完整版': 1011,
  '观看': 18240,
  '全集': 7890,
  '在线': 18840,
  '手机': 11613,
  '大全': 31519},
 '百度': {'云': 11794,
  '贴吧': 11417,
  '资源': 2717,
  '下载': 34264,
  '百科': 205,
  '分享': 317,
  '云盘': 1723,
  '地图': 2724},
 '英语': {'上册': 3351,
  '作文': 1

### 5. 计算竞争性关键字的竞争度

竞争性Comp测度的计算公式为：$$Comp_s(k,s)=\frac{|\{ka\}|}{(|\{a\}|-|\{sa\}|)}$$

在第2步中，我们得到了$|\{sa\}|$，在第四步中得到了$|\{ka\}|$, 还需要计算$|\{a\}|$

In [267]:
# 计算所有中介关键词的搜索量|{a}|
def count_midkeyword(seedword):
    count_dict=dict.fromkeys(midkeywords[seedword],0)
    query_data = open('./link_clean_data.train','r',encoding = 'utf-8')
    for sentence in query_data: 
        for midkeyword in midkeywords[seedword]:
            if midkeyword in sentence:
                count_dict[midkeyword]+=1
    query_data.close()
    return count_dict

for seedword in seedwords_list:
    a_query_volume[seedword]=count_midkeyword(seedword)

In [353]:
# 计算Comp
import copy
def getcomp():
    comp_query_volume = copy.deepcopy(sa_query_volume)
    for seedword in a_query_volume:
        for midkeyword in a_query_volume[seedword]:
             comp_query_volume[seedword][midkeyword] = ka_query_volume[seedword][midkeyword]/(a_query_volume[seedword][midkeyword]-sa_query_volume[seedword][midkeyword])
    return comp_query_volume  
result_query_volume=getcomp()
result_query_volume

关键词k与种子关键词s的竞争性程度：$$Comp(k,s)=\sum_{i=1}^{m}{\{w_{a_i}(k)\times Comp_{a_i}(k,s)\}}$$

In [374]:
def comp_result():
    result={}
    for seedword in result_query_volume:
        result.setdefault(seedword,{})
        i=0
        for midkeyword in result_query_volume[seedword]:
            result[seedword][compkeywords[seedword][i]]=w_midkeyword[seedword][midkeyword]*result_query_volume[seedword][midkeyword]
            i+=1
    return result
comp_k_s=comp_result()
#关键词k与种子关键词s的竞争性程度
comp_k_s

{'图片': {'做法': 0.013955161590091605,
  '男生': 0.002875590109732842,
  '句子': 0.004113912409570989,
  '价格表': 0.0025556670415304347,
  '视频': 0.0031160620467385425,
  '最新': 0.0004216478233177609,
  '下载': 0.0006986138243731486,
  '头像': 0.0012040801589533357},
 '手机': {'txt': 0.017798197073686906,
  '头像': 0.007079432730825036,
  '苹果7': 0.009009323476211379,
  '下载': 0.004054286840066623,
  '荣耀': 0.0060396363428516665,
  '大全': 0.000976264616998147,
  '路由器': 0.0027978694886293867,
  '高清': 0.002704297428539093},
 '小说': {'穿越火线': 0.010831361980816166,
  '电影': 0.0035587962412817024,
  '前十名': 0.004287126073027957,
  '耽美': 0.005750169297827209,
  '全文': 0.01880985188788177,
  '末世': 0.0009882396333854395,
  '女主角': 0.01510449159303849,
  'txt': 0.006966854854433018},
 '视频': {'设计': 0.00321522255715073,
  '观看': 0.01875768697772667,
  '在线': 0.023531480681324313,
  '图片': 0.0074434830493032254,
  '下载': 0.0015827318757881223,
  '游戏': 0.002081773954901387,
  '大全': 0.0024511317834844644,
  '教学': 9.720134105198065e

### 6. 竞争性关键字排序输出

In [385]:
import pandas as pd
def comp_print(seedword):
    resultpd= pd.DataFrame.from_dict(comp_k_s[seedword],orient='index',columns=['竞争度'])
    print(resultpd.sort_values(by='竞争度',ascending=False))

##### 图片的竞争性关键字

In [386]:
comp_print('图片')

          竞争度
做法   0.013955
句子   0.004114
视频   0.003116
男生   0.002876
价格表  0.002556
头像   0.001204
下载   0.000699
最新   0.000422


##### 手机的竞争性关键字

In [387]:
comp_print('手机')

          竞争度
txt  0.017798
苹果7  0.009009
头像   0.007079
荣耀   0.006040
下载   0.004054
路由器  0.002798
高清   0.002704
大全   0.000976


##### 小说的竞争性关键字

In [388]:
comp_print('小说')

           竞争度
全文    0.018810
女主角   0.015104
穿越火线  0.010831
txt   0.006967
耽美    0.005750
前十名   0.004287
电影    0.003559
末世    0.000988


##### 视频的竞争性关键字

In [389]:
comp_print('视频')

         竞争度
在线  0.023531
观看  0.018758
图片  0.007443
设计  0.003215
大全  0.002451
游戏  0.002082
下载  0.001583
教学  0.000097


##### 下载的竞争性关键字

In [390]:
comp_print('下载')

           竞争度
重生    0.016821
百度云   0.013134
阅读    0.011171
电视剧   0.007204
qq    0.004948
手机    0.004546
手机游戏  0.001978
穿越    0.001554


##### qq的竞争性关键字

In [391]:
comp_print('qq')

            竞争度
侠盗     0.014775
女生     0.014584
重生     0.008675
查询     0.007701
下载     0.006625
163邮箱  0.006128
背景音乐   0.004009
字      0.003718


##### 电影的竞争性关键字

In [392]:
comp_print('电影')

         竞争度
在线  0.032097
观看  0.015997
官网  0.011844
图片  0.009472
下载  0.006583
视频  0.004359
中国  0.002500
qq  0.002070


##### 百度竞争性关键字

In [393]:
comp_print('百度')

            竞争度
青云志    0.074163
贴吧     0.062961
人力资源   0.023117
360云盘  0.021024
txt    0.012546
中国地图   0.004419
视频     0.004219
糗事     0.003775


##### 英语的竞争性关键字

In [394]:
comp_print('英语')

           竞争度
英文单词  0.026610
语文    0.018246
字     0.014916
阅读    0.012619
英文翻译  0.011099
成绩    0.006791
大学生   0.003794
视频教程  0.003636


##### 游戏的竞争性关键字

In [395]:
comp_print('游戏')

            竞争度
单机版    0.024102
txt    0.021472
图片     0.017431
地方     0.013228
qq     0.007953
大全     0.005983
笔记本电脑  0.003804
exo    0.003732
